# Build Classification Models

In [50]:
import pandas as pd
cuisines_df = pd.read_csv("../data/cleaned_cuisines.csv")
cuisines_df.head()

,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report, precision_recall_curve
from sklearn.svm import SVC
import numpy as np

接下来需要将数据分为训练模型所需的 X（译者注：代表特征数据）和 y（译者注：代表标签数据）两个 dataframe。首先可将 cuisine 列的数据单独保存为的一个 dataframe 作为标签（label）。

In [52]:
cuisines_label_df = cuisines_df['cuisine']
cuisines_label_df.head()

0    indian
1    indian
2    indian
3    indian
4    indian
Name: cuisine, dtype: object

调用 drop() 方法将 Unnamed: 0 和 cuisine 列删除，并将余下的数据作为可以用于训练的特证（feature）数据:

In [53]:
cuisines_feature_df = cuisines_df.drop(['Unnamed: 0', 'cuisine'], axis=1)
cuisines_feature_df.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### 选择分类器
线性模型（Linear Models）
支持向量机（Support Vector Machines）
随机梯度下降（Stochastic Gradient Descent）
最近邻（Nearest Neighbors）
高斯过程（Gaussian Processes）
决策树（Decision Trees）
集成方法（投票分类器）（Ensemble methods（voting classifier））
多类别多输出算法（多类别多标签分类，多类别多输出分类）（Multiclass and multioutput algorithms (multiclass and multilabel classification, multiclass-multioutput classification)）

### 如何选择分类器?
可以选择多个分类器，然后进行比较，参照sk-learn的[脚本](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)

Azure [AutoML](https://learn.microsoft.com/zh-cn/training/modules/automate-model-selection-with-azure-automl/?WT.mc_id=academic-77952-leestott) 提供云端运行这些算法并比较的方案

[机器学习速查表](https://learn.microsoft.com/zh-cn/azure/machine-learning/algorithm-cheat-sheet?WT.mc_id=academic-77952-leestott#how-to-use-the-machine-learning-algorithm-cheat-sheet)

通过逻辑回归算法，来演练一下如何进行你的第一个机器学习模型的训练。

当我们需要 Scikit-learn 进行逻辑回归运算时，multi_class 以及 solver是最重要的两个参数，因此我们需要特别说明一下。 multi_class 是分类方式选择参数，而solver优化算法选择参数。值得注意的是，并不是所有的 solvers 都可以与multi_class参数进行匹配的。

根据官方文档，在多类型分类问题中:
- 当 multi_class 被设置为 ovr 时，将使用 “一对其余”(OvR)策略（scheme）。
- 当 multi_class 被设置为 multinomial 时，则使用的是交叉熵损失（cross entropy loss） 作为损失函数。(注意，目前multinomial只支持‘lbfgs’, ‘sag’, ‘saga’以及‘newton-cg’等 solver 作为损失函数的优化方法)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(cuisines_feature_df, cuisines_label_df, test_size=0.3)

In [59]:
lr = LogisticRegression(multi_class='ovr',solver='liblinear')
model = lr.fit(X_train.values, np.ravel(y_train))

accuracy = model.score(X_test.values, y_test)
print ("Accuracy is {}".format(accuracy))

Accuracy is 0.79232693911593


✅ 也可以试试其他 solver 比如 lbfgs, 这也是默认参数
> 用 Pandas 的 ravel 方法可以在需要的时候将你的数据进行降维

你也可以通过查看某一行数据（比如第 50 行）来观测到模型运行的情况:

[X_test.iloc[50] != 0]：这是一个布尔条件表达式，用于筛选出第50个样本中非零的特征。条件X_test.iloc[50] != 0会生成一个布尔类型的Series对象，其中每个元素表示对应特征是否非零。

.keys()函数被应用于这个筛选后的Series对象，以获取非零特征的键（即列名）。

In [56]:
print(f'ingredients: {X_test.iloc[50][X_test.iloc[50]!=0].keys()}')
print(f'cuisine: {y_test.iloc[50]}')

ingredients: Index(['cane_molasses', 'mango', 'meat', 'onion', 'orange', 'soy_sauce',
       'star_anise'],
      dtype='object')
cuisine: chinese


`.values:` 将获取到的第50个样本转换为一个NumPy数组。

`.reshape(-1, 1)`：`reshape()`函数将一维数组转换为二维数组。其中，参数-1表示根据数组的大小自动确定数组的维度，而1表示希望将数组转换为1列。

`model.predict_proba()`：该方法可以返回测试数据的类别概率预测结果。

`model.classes_`：这是模型对象（model）的一个属性，用于获取模型的类别信息。它返回一个数组，其中包含模型预测的所有类别。

`by=[0]`：这是`sort_values()`方法的参数，指定按照哪一列或索引进行排序。在这里，我们指定使用索引为0的列进行排序，即第一列。

`ascending=[False]`：这是`sort_values()`方法的参数，指定排序的顺序。[False]表示降序排列。

In [60]:
test= X_test.iloc[50].values.reshape(-1, 1).T
proba = model.predict_proba(test)
classes = model.classes_
resultdf = pd.DataFrame(data=proba, columns=classes)

topPrediction = resultdf.T.sort_values(by=[0], ascending = [False])
topPrediction.head()

,0
chinese,0.818790
korean,0.092443
thai,0.038051
indian,0.033608
japanese,0.017108


In [62]:
y_pred = model.predict(X_test.values)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     chinese       0.71      0.71      0.71       231
      indian       0.88      0.89      0.88       202
    japanese       0.75      0.74      0.75       257
      korean       0.86      0.77      0.81       272
        thai       0.76      0.86      0.81       237

    accuracy                           0.79      1199
   macro avg       0.80      0.80      0.79      1199
weighted avg       0.79      0.79      0.79      1199

